<a href="https://colab.research.google.com/github/LegionXF23/DevSoc-Assign/blob/main/Copy_of_LLM_Response_Generator_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import requests
import time
from datetime import datetime

# --- Configuration ---
# NOTE: The API key is usually loaded from an environment variable for security.
# For this demonstration, leave it as an empty string. The platform will handle it.
API_KEY = ""
API_URL_TEMPLATE = "https://generativelanguage.googleapis.com/v1beta/models/{model}:generateContent?key={key}"
MODEL_NAME = "gemini-2.5-flash-preview-09-2025"

INPUT_FILE_NAME = "input_prompts.txt"
OUTPUT_FILE_PREFIX = "llm_responses_"
MAX_RETRIES = 5
INITIAL_DELAY = 2  # seconds
# ---------------------

def call_gemini_api(prompt, model=MODEL_NAME):
    """
    Makes a POST request to the Gemini API with exponential backoff.
    Returns the generated text or None on persistent failure.
    """
    url = API_URL_TEMPLATE.format(model=model, key=API_KEY)

    # Payload for the API request
    payload = {
        "contents": [{"parts": [{"text": prompt}]}],
        # Using Google Search grounding for up-to-date information
        "tools": [{"google_search": {}}],
        "systemInstruction": {
            "parts": [{
                "text": "You are a helpful, professional, and concise assistant. Answer the user's query directly."
            }]
        }
    }

    delay = INITIAL_DELAY

    for attempt in range(MAX_RETRIES):
        try:
            print(f"-> Attempting to call API for prompt: '{prompt[:50]}...' (Attempt {attempt + 1}/{MAX_RETRIES})")

            response = requests.post(
                url,
                headers={'Content-Type': 'application/json'},
                data=json.dumps(payload),
                timeout=30  # Timeout after 30 seconds
            )

            # Raise an HTTPError for bad responses (4xx or 5xx)
            response.raise_for_status()

            # Parse the JSON response
            result = response.json()
            candidate = result.get('candidates', [{}])[0]

            # Extract the text content
            generated_text = candidate.get('content', {}).get('parts', [{}])[0].get('text')

            if generated_text:
                print("<- Success.")
                return generated_text
            else:
                # Handle cases where the API returns a success status but no text content
                print("<- API returned successfully, but no text was generated. Retrying.")

        except requests.exceptions.RequestException as e:
            # Handle network errors, timeouts, or bad HTTP status codes
            print(f"<- Error on attempt {attempt + 1}: {e}")

        # If this was the last attempt, break the loop
        if attempt < MAX_RETRIES - 1:
            print(f"Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2 # Exponential backoff
        else:
            print("<- Max retries reached. Failing this prompt.")
            return None

    return None

def process_prompts(input_file):
    """Reads prompts from the input file and processes them via the LLM API."""
    if not os.path.exists(input_file):
        print(f"Error: Input file '{input_file}' not found.")
        return []

    results = []

    # Read prompts, stripping empty lines and whitespace
    with open(input_file, 'r', encoding='utf-8') as f:
        prompts = [line.strip() for line in f if line.strip()]

    total_prompts = len(prompts)
    print(f"\nFound {total_prompts} prompts to process.")

    for i, prompt in enumerate(prompts):
        print(f"\n--- Processing Prompt {i + 1}/{total_prompts} ---")

        # Get the LLM response
        response_text = call_gemini_api(prompt)

        # Store the result object
        results.append({
            "id": i + 1,
            "prompt": prompt,
            "response": response_text if response_text else "API Call Failed or returned no content."
        })

    return results

def save_results(data, prefix):
    """Saves the list of results to a JSON file with a timestamp."""
    # Generate a timestamped file name (YYYYMMDD_HHMMSS)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"{prefix}{timestamp}.json"

    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"\n--- Process Complete ---")
        print(f"Successfully saved {len(data)} results to: {output_filename}")
        return output_filename
    except Exception as e:
        print(f"Error saving file: {e}")
        return None

if __name__ == "__main__":
    print(f"--- LLM Response Generator using {MODEL_NAME} ---")

    # 1. Process prompts and get responses
    response_data = process_prompts(INPUT_FILE_NAME)

    # 2. Save the results
    if response_data:
        save_results(response_data, OUTPUT_FILE_PREFIX)